In [23]:
from pyomo.environ import *
from pyomo import *
import pandas as pd
import math
from numpy import inf
import heapq
import logging
from pyomo.environ import *

# Set the logging level to ERROR to suppress infeasibility warnings
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
model = ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
model.I = Set(initialize=range(1, 8))  # Set of orders i
model.J = Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
model.r = Param(model.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
model.d = Param(model.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
model.c = Param(model.I, model.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1

model.p = Param(model.I, model.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
                                               (2, 1): 6, (2, 2): 8, (2, 3): 7,
                                               (3, 1): 11, (3, 2): 16, (3, 3): 13,
                                               (4, 1): 6, (4, 2): 12, (4, 3): 8,
                                               (5, 1): 10, (5, 2): 16, (5, 3): 12,
                                               (6, 1): 7, (6, 2): 12, (6, 3): 10,
                                               (7, 1): 10, (7, 2): 8, (7, 3): 10})

# Uncomment the following and adjust the values for the second set of durations p_im for Set 2
'''
model.p = Param(model.I, model.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,
                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,
                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,
                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,
                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,
                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,
                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})
'''

'\nmodel.p = Param(model.I, model.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,\n                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,\n                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,\n                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,\n                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,\n                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,\n                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})\n'

In [24]:
#Define Variables
model.x = Var(model.I,model.J,domain = NonNegativeReals,bounds = (0,1))
model.ts = Var(model.I,domain = NonNegativeReals)
model.y = Var(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i != i_prime]),domain = NonNegativeReals,bounds = (0,1))

In [25]:
#Define constraints
model.earliest_start =  Constraint(model.I, rule=lambda model, i: 
        model.ts[i] >= model.r[i])
model.latest_start =  Constraint(model.I, rule=lambda model, i: 
        model.ts[i] <= model.d[i] - sum(model.p[i,m]*model.x[i,m] for m in model.J))
model.assing_to_one_unit =  Constraint(model.I, rule=lambda model, i: 
        sum(model.x[i,m] for m in model.J)==1)
model.tightening_constraint =  Constraint(model.J, rule=lambda model, m: 
        sum(model.p[i,m]*model.x[i,m] for i in model.I)<=max(model.d[i] for i in model.I)-min(model.r[i] for i in model.I))
model.precedence_constraint =  Constraint(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i > i_prime]), model.J, rule=lambda model,i,i_prime,m: 
        model.y[i,i_prime]+model.y[i_prime,i] >= model.x[i,m]+model.x[i_prime,m]-1 )
model.M = 1000
model.sequencing_constraint =  Constraint(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i != i_prime]), rule=lambda model, i,i_prime: 
        model.ts[i_prime] >= model.ts[i]+sum(model.p[i,m]*model.x[i,m] for m in model.J)-model.M*(1-model.y[i,i_prime]))


#model.precedence_constraint2 =  Constraint(Set(initialize=[(i, i_prime,m,m_prime) for i in model.I for i_prime in model.I for m in model.J for m_prime in model.J if i > i_prime and m!=m_prime]),  rule=lambda model,i,i_prime,m,m_prime: 
 #      model.y[i,i_prime]+model.y[i_prime,i]+model.x[i,m]+model.x[i_prime,m_prime] <= 2 )

#Define objective
model.cost = Objective(sense=minimize, expr = sum(model.c[i,m]*model.x[i,m] for i in model.I for m in model.J))



In [26]:
model.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

4 Param Declarations
    c : Size=21, Index=I*J, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :    10
        (1, 2) :     6
        (1, 3) :     8
        (2, 1) :     8
        (2, 2) :     5
        (2, 3) :     6
        (3, 1) :    12
        (3, 2) :     7
        (3, 3) :    10
        (4, 1) :    10
        (4, 2) :     6
        (4, 3) :     8
        (5, 1) :     8
        (5, 2) :     5
        (5, 3) :     7
        (6, 1) :    12
        (6, 2) :     7
        (6, 3) :    10
        (7, 1) :    12
        (7, 2) :     7
        (7, 3) :    10
    d : Size=7, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
    

In [27]:
#We define a node of the branch and bound as a class
class Node:
   def __init__(self, name,df_xfix,df_yfix):
      self.left = None #left branching node
      self.right = None #right branchingnots
      self.name = name
      self.df_xfix = df_xfix #Dataframe storing the value of the x variables to be fixed in the node. If not fixed, the corresponding value is Nan
      self.df_yfix = df_yfix #Dataframe storing the value of the y variables to be fixed in the node. If not fixed, the corresponding value is Nan
      for i in model.x:
          model.x[i].unfix()
      for i in model.y:
          model.y[i].unfix()
      for i in model.x:
          if(math.isnan(df_xfix.at[i[0],i[1]])==False):
              model.x[i].fix(df_xfix.at[i])
              #print(df_xfix.at[i])
      for i in model.y:
          if(math.isnan(df_yfix.at[i[0],i[1]])==False):
              model.y[i].fix(df_yfix.at[i])
      solver = SolverFactory('gurobi')
      results = solver.solve(model)
      self.status = "Infeasible/unbounded" #Status for solving the corresponding LP for the node
      self.df_xsol = None #Dataframe with x variable value for solving the corresponding LP for the node
      self.df_ysol = None #Dataframe with y variable value for solving the corresponding LP for the node
      self.LB = None #cost for solving the corresponding LP for the node/Lower bound of the node
      if results.solver.termination_condition in {TerminationCondition.optimal}:
           self.status = "Feasible"
           df_xsol = pd.DataFrame(columns=model.J, index=model.I)
           df_ysol = pd.DataFrame(columns=model.I, index=model.I)
           for i in model.I:
               for j in model.J:
                   df_xsol.at[i,j] = model.x[i,j]()
           for i in model.I:
               for j in model.I:
                   if(i!=j):
                       df_ysol.at[i,j] = model.y[i,j]()
           self.df_xsol = df_xsol
           self.df_ysol = df_ysol
           self.LB = model.cost()


   def __lt__(self, other):
       return self.LB < other.LB #To set priority of nodes
      

In [28]:
#Heuristic for upper bound
def upper_bound(m,df_xfix,df_yfix,df_xsol,df_ysol):
    UB = float('inf')
    solver = SolverFactory('gurobi')
    for i in m.I:
        max_i = 1
        c = 0            
        for j in m.J:
            if(df_xsol.at[i,j] > df_xsol.at[i,max_i]):
              max_i = j
        m.x[i,max_i].fix(1.0)
        for j in m.J:
          if(j != max_i):
            m.x[i,j].fix(0.0)
    for i in m.y:
      m.y[i].fix(float(round(df_ysol.at[i]))) 
    solver = SolverFactory('gurobi')
    results = solver.solve(m)
    
    df_xsol1 = pd.DataFrame(columns=m.J, index=m.I)
    df_ysol1 = pd.DataFrame(columns=m.I, index=m.I)
    if results.solver.termination_condition in {TerminationCondition.optimal}:
      UB = m.cost()
      for i in m.x:
        df_xsol1.at[i] = m.x[i]() 
      for i in m.y:
        df_ysol1.at[i] = m.y[i]() 
    for i in m.I:
        for j in m.J:
            #if(math.isnan(df_xfix[i,j])==False):
            m.x[i,j].unfix()
    for i in m.y:
        #if(math.isnan(df_yfix[i[0],i[1]])==False):
        m.y[i].unfix()
        

    return UB,df_xsol1,df_ysol1

In [29]:
#Branch and bound with most fractional variable branching rule
def branch_and_bound_frac(m,heap):
    global  UB_universal
    global df_unx
    global df_uny
    global nodes_total
    global nodes_feasible
    while(len(heap)>0):
        Tree = heapq.heappop(heap)
        print(len(heap))
        print(Tree.name)
        if (Tree.LB > UB_universal):
            print(Tree.name) #Prune by optimzality
            continue
        #Checking the integrality 
        integer_feasible = True
        for v in m.x:
            if not Tree.df_xsol.at[v].is_integer():
                integer_feasible = False
        for v in m.y:
            if not Tree.df_ysol.at[v].is_integer():
                integer_feasible = False
        print(integer_feasible)
        if integer_feasible == True:
            print(Tree.name)
            if(UB_universal>Tree.LB):
                UB_universal = Tree.LB
                df_unx = Tree.df_xsol
                df_uny = Tree.df_ysol
        
            continue

        #Choosing which variable to branch on
        var_to_branch = (1,1)
        var_name = "x(1,1)"
        fractionality_of_var_to_branchdiff = 0
        frac = 0
        if(Tree.df_xsol.at[1,1]>=0.5):
            frac = 1 - Tree.df_xsol.at[1,1]
        else:
            frac = Tree.df_xsol.at[1,1]
        fractionality_of_var_to_branchdiff = 0.5-frac
            
        for i in m.x:
            frac = 0
            if(Tree.df_xsol.at[i]>=0.5):
                frac = 1 - Tree.df_xsol.at[i]
            else:
                frac = Tree.df_xsol.at[i]
            if(fractionality_of_var_to_branchdiff>0.5-frac):
                fractionality_of_var_to_branchdiff = 0.5-frac
                var_to_branch = i
                var_name = "x"+str(i)
        for i in m.y:
            frac = 0
            if(Tree.df_ysol.at[i]>=0.5):
                frac = 1 - Tree.df_ysol.at[i]
            else:
                frac = Tree.df_ysol.at[i]
            if(fractionality_of_var_to_branchdiff>0.5-frac):
                fractionality_of_var_to_branchdiff = 0.5-frac
                var_to_branch = i
                var_name = "y"+str(i)
        UB,df_xsol,df_ysol = upper_bound(m,Tree.df_xfix,Tree.df_yfix,Tree.df_xsol,Tree.df_ysol)
        if(UB<UB_universal):
            UB_universal = UB
            df_unx = df_xsol
            df_uny = df_ysol
        df_lxfix = pd.DataFrame(columns=model.J, index=model.I)
        df_rxfix = pd.DataFrame(columns=model.J, index=model.I)
        df_lyfix = pd.DataFrame(columns=model.I, index=model.I)
        df_ryfix = pd.DataFrame(columns=model.I, index=model.I)
        for i in m.x:
            df_lxfix.at[i] = Tree.df_xfix.at[i]
            df_rxfix.at[i] = Tree.df_xfix.at[i]
        for i in m.y:
            df_lyfix.at[i] = Tree.df_yfix.at[i]
            df_ryfix.at[i] = Tree.df_yfix.at[i]
        #print(df_lxfix)
        if("x" in var_name):
            df_lxfix.at[var_to_branch] = 0.0
            df_rxfix.at[var_to_branch] = 1.0
        if("y" in var_name):
            df_lyfix.at[var_to_branch] = 0.0
            df_ryfix.at[var_to_branch] = 1.0
        #print(df_lxfix)
        #New nodes
        nodes_total = nodes_total+2
        Tree.left = Node(Tree.name+", "+var_name+"=0",df_lxfix,df_lyfix)
        Tree.right = Node(Tree.name+", "+var_name+"=1",df_rxfix,df_ryfix)
        #We prune the node if infeasible
        if(Tree.left.status == "Feasible"):
          if(Tree.left.LB<=UB_universal):
            #print(Tree.left.status)
            nodes_feasible = nodes_feasible+1
            heapq.heappush(heap,Tree.left)
        #print(list_xy)
        if(Tree.right.status == "Feasible"):
          if(Tree.right.LB<=UB_universal):
            #print(Tree.left.status)
            nodes_feasible = nodes_feasible+1
            heapq.heappush(heap,Tree.right)




In [30]:
def branch_and_bound_strong(m,heap):
    global  UB_universal
    global df_unx
    global df_uny
    global nodes_total
    global nodes_feasible
    while(len(heap)>0):
        Tree = heapq.heappop(heap)
        print(len(heap))
        print(Tree.name)
        if (Tree.LB >= UB_universal):
            print(Tree.name) #Prune by optimality
            continue
        #Checking integrality
        integer_feasible = True
        var_to_branch = (1,1)
        var_name = "x(1,1)"
        for v in m.x:
            if not Tree.df_xsol.at[v].is_integer():
                integer_feasible = False
                var_to_branch = v
                var_name = "x"+str(v)
        for v in m.y:
            if not Tree.df_ysol.at[v].is_integer():
                integer_feasible = False
                var_to_branch = v
                var_name = "y"+str(v)
        print(integer_feasible)
        if integer_feasible == True:
            print(Tree.name)
            if(UB_universal>Tree.LB):
                UB_universal = Tree.LB
                df_unx = Tree.df_xsol
                df_uny = Tree.df_ysol
        
            continue
        #Strong branching
        branch_change = Tree.LB
        for i in m.x:
            if(math.isnan(df_xfix.at[i[0],i[1]])==False):
                m.x[i].fix(df_xfix.at[i])
              #print(df_xfix.at[i])
        for i in model.y:
            if(math.isnan(df_yfix.at[i[0],i[1]])==False):
                m.y[i].fix(df_yfix.at[i])
        for v in m.x:
            if not (Tree.df_xsol.at[v].is_integer() or math.isnan(df_xfix.at[v])==False):
                a = 0
                b = 0
                m.x[v].fix(0.0)
                #print("x"+str(v))
                #print(Tree.df_xsol.at[v])
                solver = SolverFactory('gurobi')
                results = solver.solve(model)
                if results.solver.termination_condition in {TerminationCondition.optimal}:
                    a = m.cost()
                    m.x[v].unfix()
                    m.x[v].fix(1.0)
                    solver = SolverFactory('gurobi')
                    results = solver.solve(model)
                    if results.solver.termination_condition in {TerminationCondition.optimal}:
                        b = model.cost()
                        m.x[v].unfix()
                        if(min(a,b)>=branch_change):
                            branch_change = min(a,b)
                            var_to_branch = v
                            var_name = "x"+str(v)
                    else:
                        a = m.cost()
                        m.x[v].unfix()
                        if(a>=branch_change):
                            branch_change = a
                            var_to_branch = v
                            var_name = "x"+str(v)
                else:
                    m.x[v].unfix()
                    m.x[v].fix(1.0)
                    solver = SolverFactory('gurobi')
                    results = solver.solve(model)
                    if results.solver.termination_condition in {TerminationCondition.optimal}:
                        b = model.cost()
                        m.x[v].unfix()
                        if(b>=branch_change):
                            branch_change = b
                            var_to_branch = v
                            var_name = "x"+str(v)
                            
                    else:
                        m.x[v].unfix()
                        #continue
                #print("x"+str(v))
                #print(Tree.LB)
                #print(a)
                #print(b)
        for v in m.y:
            if not (Tree.df_ysol.at[v].is_integer() or math.isnan(df_yfix.at[v])==False):
                #print(branch_change)
                a = 0
                b = 0
                m.y[v].fix(0.0)
                #print("y"+str(v))
                #print(Tree.df_ysol.at[v])
                solver = SolverFactory('gurobi')
                results = solver.solve(model)
                if results.solver.termination_condition in {TerminationCondition.optimal}:
                    a = model.cost()
                    m.y[v].unfix()
                    m.y[v].fix(1.0)
                    solver = SolverFactory('gurobi')
                    results = solver.solve(model)
                    if results.solver.termination_condition in {TerminationCondition.optimal}:
                        b = m.cost()
                        m.y[v].unfix()
                        if(min(a,b)>=branch_change):
                            branch_change = min(a,b)
                            var_to_branch = v
                            var_name = "y"+str(v)
                    else:
                        a = m.cost()
                        model.y[v].unfix()
                        if(a>=branch_change):
                            branch_change = a
                            var_to_branch = v
                            var_name = "y"+str(v)
                else:
                    m.y[v].unfix()
                    m.y[v].fix(1.0)
                    solver = SolverFactory('gurobi')
                    results = solver.solve(model)
                    if results.solver.termination_condition in {TerminationCondition.optimal}:
                        b = m.cost()
                        m.y[v].unfix()
                        if(b>=branch_change):
                            branch_change = b
                            var_to_branch = v
                            var_name = "y"+str(v)
                            
                    else:
                        model.y[v].unfix()
                        #continue 
                #print("y"+str(v))
                #print(Tree.LB)
                #print(a)
                #print(b)
                
                
    
        print(branch_change)
        print(var_to_branch)
        print(Tree.LB)
        UB,df_xsol,df_ysol = upper_bound(m,Tree.df_xfix,Tree.df_yfix,Tree.df_xsol,Tree.df_ysol)
        if(UB<UB_universal):
            UB_universal = UB
            df_unx = df_xsol
            df_uny = df_ysol
        df_lxfix = pd.DataFrame(columns=model.J, index=model.I)
        df_rxfix = pd.DataFrame(columns=model.J, index=model.I)
        df_lyfix = pd.DataFrame(columns=model.I, index=model.I)
        df_ryfix = pd.DataFrame(columns=model.I, index=model.I)
        for i in m.x:
            df_lxfix.at[i] = Tree.df_xfix.at[i]
            df_rxfix.at[i] = Tree.df_xfix.at[i]
        for i in m.y:
            df_lyfix.at[i] = Tree.df_yfix.at[i]
            df_ryfix.at[i] = Tree.df_yfix.at[i]
        #print(df_lxfix)
        if("x" in var_name):
            df_lxfix.at[var_to_branch] = 0.0
            df_rxfix.at[var_to_branch] = 1.0
        if("y" in var_name):
            df_lyfix.at[var_to_branch] = 0.0
            df_ryfix.at[var_to_branch] = 1.0
        #print(df_lxfix)
        nodes_total = nodes_total+2
        Tree.left = Node(Tree.name+", "+var_name+"=0",df_lxfix,df_lyfix)
        Tree.right = Node(Tree.name+", "+var_name+"=1",df_rxfix,df_ryfix)
        #We prune the node if infeasible
        if(Tree.left.status == "Feasible"):
          if(Tree.left.LB<=UB_universal):
            nodes_feasible = nodes_feasible+1
            #print(Tree.left.status)
            heapq.heappush(heap,Tree.left)
        #print(list_xy)
        if(Tree.right.status == "Feasible"):
          if(Tree.right.LB<=UB_universal):
            nodes_feasible = nodes_feasible+1
            #print(Tree.left.status)
            heapq.heappush(heap,Tree.right)
    #m.pprint()
    
    #m.pprint()

In [31]:
df_xfix = pd.DataFrame(columns=model.J, index=model.I)
df_yfix = pd.DataFrame(columns=model.I, index=model.I)
global nodes_total
global nodes_feasible
nodes_total = 0
nodes_feasible = 0

global UB_universal 
UB_universal = float(inf)
Tree = Node("root",df_xfix,df_yfix)
df_unx = pd.DataFrame(columns=model.J, index=model.I)
df_uny = pd.DataFrame(columns=model.I, index=model.I)    
for i in model.x:
    var = model.x[i]
    var.domain = NonNegativeReals
    var.bounds = (0,1)
for i in model.y:
    var = model.y[i]
    var.domain = NonNegativeReals
    var.bounds = (0,1)
heap = [Tree]
if(Tree.status == "Feasible"):
    nodes_total = 1
    nodes_feasible = 1
branch_and_bound_frac(model,heap)

0
root
False
1
root, x(3, 1)=0
False
2
root, x(3, 1)=0, x(3, 2)=0
False
3
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0
False
4
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=1
False
4
root, x(3, 1)=1
False
5
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=0
False
6
root, x(3, 1)=1, x(4, 3)=1
False
7
root, x(3, 1)=0, x(3, 2)=1
False
8
root, x(3, 1)=1, x(4, 3)=1, x(1, 2)=0
False
9
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=1, x(1, 3)=0
False
9
root, x(3, 1)=0, x(3, 2)=1, x(1,1)=0
False
10
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=1, x(1, 3)=0, x(5, 3)=0
False
10
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=0, x(1,1)=0
False
11
root, x(3, 1)=0, x(3, 2)=1, x(1,1)=1
False
12
root, x(3, 1)=1, x(4, 3)=0
False
13
root, x(3, 1)=1, x(4, 3)=0, x(6, 3)=0
False
14
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=0, x(1,1)=1
False
15
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=0, x(1,1)=1, x(6, 3)=0
False
16
root, x(3, 1)=0, x(3, 2)=0, x(1, 2)=0, x(4, 3)=0, x(1,1)=0, x(2, 2)=0
False
17
root, x(3

In [32]:
print(UB_universal)
print(df_unx)
print(df_uny)
print(nodes_total)
print(nodes_feasible)  #Problem set 1 : frac

60.0
     1    2    3
1  0.0  1.0  0.0
2  1.0  0.0  0.0
3  1.0  0.0  0.0
4  1.0  0.0  0.0
5  0.0  0.0  1.0
6  0.0  1.0  0.0
7  0.0  0.0  1.0
     1    2    3    4    5    6    7
1  NaN  0.0  0.0  0.0  0.0  1.0  0.0
2  0.0  NaN  1.0  1.0  0.0  1.0  0.0
3  0.0  0.0  NaN  1.0  0.0  0.0  0.0
4  0.0  0.0  0.0  NaN  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  NaN  0.0  0.0
6  0.0  0.0  0.0  0.0  0.0  NaN  0.0
7  0.0  0.0  0.0  0.0  1.0  0.0  NaN
1455
777
